In [2]:
import os
import pandas as pd
import numpy as np
import findspark
findspark.init('/home/soyoung/standaloneSpark/spark-2.2.1-bin-hadoop2.7')

from pyspark.sql import SparkSession

In [3]:
%%time
spark = SparkSession.builder.appName('ops').getOrCreate()

CPU times: user 12 ms, sys: 12 ms, total: 24 ms
Wall time: 3.71 s


In [4]:
path='./Python-and-Spark-for-Big-Data-master/Spark_DataFrames/'

In [5]:
%%time
df = spark.read.csv(path+'appl_stock.csv', inferSchema=True, header=True)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.37 s


In [8]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [9]:
df.head(3)

[Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039),
 Row(Date=datetime.datetime(2010, 1, 5, 0, 0), Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002),
 Row(Date=datetime.datetime(2010, 1, 6, 0, 0), Open=214.379993, High=215.23, Low=210.750004, Close=210.969995, Volume=138040000, Adj Close=27.333178000000004)]

In [18]:
df.filter('Open>650').select(['Date','Volume','Low']).show()

+-------------------+---------+-----------------+
|               Date|   Volume|              Low|
+-------------------+---------+-----------------+
|2012-08-20 00:00:00|153346200|       649.900002|
|2012-08-21 00:00:00|203179900|       650.330025|
|2012-08-22 00:00:00|141330700|       648.110001|
|2012-08-23 00:00:00|105032200|       661.149979|
|2012-08-24 00:00:00|109335100|       655.550011|
|2012-08-27 00:00:00|106752100|673.5400089999999|
|2012-08-28 00:00:00| 66854200|       670.669983|
|2012-08-29 00:00:00| 50701700|       672.600014|
|2012-08-30 00:00:00| 75674900|       662.849991|
|2012-08-31 00:00:00| 84580300|       657.250023|
|2012-09-04 00:00:00| 91973000|664.5000150000001|
|2012-09-05 00:00:00| 84093800|       669.599998|
|2012-09-06 00:00:00| 97799100|       670.800026|
|2012-09-07 00:00:00| 82416600|        675.77002|
|2012-09-10 00:00:00|121999500|       662.100014|
|2012-09-11 00:00:00|125995800|       656.499992|
|2012-09-12 00:00:00|178058300|       656.000008|


In [19]:
df.filter(df['close']<500).select(['Volume', 'Open','Close']).show()

+---------+------------------+------------------+
|   Volume|              Open|             Close|
+---------+------------------+------------------+
|123432400|        213.429998|        214.009998|
|150476200|        214.599998|        214.379993|
|138040000|        214.379993|        210.969995|
|119282800|            211.75|            210.58|
|111902700|        210.299994|211.98000499999998|
|115557400|212.79999700000002|210.11000299999998|
|148614900|209.18999499999998|        207.720001|
|151473000|        207.870005|        210.650002|
|108223500|210.11000299999998|            209.43|
|148516900|210.92999500000002|            205.93|
|182501900|        208.330002|        215.039995|
|153038200|        214.910006|            211.73|
|152038600|        212.079994|        208.069996|
|220441900|206.78000600000001|            197.75|
|266424900|202.51000200000001|        203.070002|
|466777500|205.95000100000001|        205.940001|
|430642100|        206.849995|        207.880005|


In [32]:
df.filter((df['Close']<500) & ~(df['Open']<500)).select(['Volume', 'Open','Close']).show()

+---------+----------+------------------+
|   Volume|      Open|             Close|
+---------+----------+------------------+
|376530000|514.259995|        497.669975|
| 59091900|500.250008|495.26997400000005|
|185798900|506.199997|494.63999900000005|
|169625400|502.539993|        499.779984|
+---------+----------+------------------+



In [36]:
r = df.filter((df['Close']<500) & (df['Open']>500)).collect()
r

[Row(Date=datetime.datetime(2012, 2, 15, 0, 0), Open=514.259995, High=526.290016, Low=496.88998399999997, Close=497.669975, Volume=376530000, Adj Close=64.477899),
 Row(Date=datetime.datetime(2013, 9, 5, 0, 0), Open=500.250008, High=500.67997699999995, Low=493.63997699999993, Close=495.26997400000005, Volume=59091900, Adj Close=65.977837),
 Row(Date=datetime.datetime(2013, 9, 10, 0, 0), Open=506.199997, High=507.450012, Low=489.500015, Close=494.63999900000005, Volume=185798900, Adj Close=65.893915),
 Row(Date=datetime.datetime(2014, 1, 30, 0, 0), Open=502.539993, High=506.49997699999994, Low=496.70002, Close=499.779984, Volume=169625400, Adj Close=66.967353)]

In [37]:
r[0]

Row(Date=datetime.datetime(2012, 2, 15, 0, 0), Open=514.259995, High=526.290016, Low=496.88998399999997, Close=497.669975, Volume=376530000, Adj Close=64.477899)

In [38]:
r[0].asDict()

{'Adj Close': 64.477899,
 'Close': 497.669975,
 'Date': datetime.datetime(2012, 2, 15, 0, 0),
 'High': 526.290016,
 'Low': 496.88998399999997,
 'Open': 514.259995,
 'Volume': 376530000}

In [39]:
type(r[0])

pyspark.sql.types.Row

In [40]:
print dir(r[0])

['__add__', '__call__', '__class__', '__contains__', '__delattr__', '__dict__', '__doc__', '__eq__', '__fields__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getnewargs__', '__getslice__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'asDict', 'count', 'index']
